# This is where we will do the initial data cleaning for the dataset.

In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
filepath = '/Users/andrewhlavacek/Documents/EDA_Netflix/netflix1.csv'

df = pd.read_csv(filepath)

# Display the first few rows of the Dataframe to show that it loaded correctly
print(df.head(10))

# First we check for any missing values in the DataFrame
print(df.isnull().sum())





  show_id     type                             title             director  \
0      s1    Movie              Dick Johnson Is Dead      Kirsten Johnson   
1      s3  TV Show                         Ganglands      Julien Leclercq   
2      s6  TV Show                     Midnight Mass        Mike Flanagan   
3     s14    Movie  Confessions of an Invisible Girl        Bruno Garotti   
4      s8    Movie                           Sankofa         Haile Gerima   
5      s9  TV Show     The Great British Baking Show      Andy Devonshire   
6     s10    Movie                      The Starling       Theodore Melfi   
7    s939    Movie   Motu Patlu in the Game of Zones          Suhas Kadav   
8     s13    Movie                      Je Suis Karl  Christian Schwochow   
9    s940    Movie          Motu Patlu in Wonderland          Suhas Kadav   

          country date_added  release_year rating   duration  \
0   United States  9/25/2021          2020  PG-13     90 min   
1          France  9/24/